# Imports

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime as dt
from datetime import timedelta as td
from pathlib import Path
from pytz import timezone
from hbt_tools import utils_hbt as uh
from hbt_tools import utils as ut
from hbt_tools import plots as hp

# Parameters

In [ ]:
save_dir = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.35-Hagnau\05 Berechnungen Grundlagen\Datenauswertung\Plots")

mes_dict = {
    'Trübung Zulauf': {
        'id': 	1077,
        'units': 'NTU',
        'yaxis': 'y2',
    },

    'Trübung Ablauf': {
        'id': 	1081,
        'units': 'NTU',
        'yaxis': 'y2',
    },

    'Niveau Retentionssickerbecken': {
        'id': 	1082,
        'units': 'cm',
        'yaxis': 'y3',
    }
}

# Read Rain

In [ ]:
path_dir = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.35-Hagnau\05 Berechnungen Grundlagen\Regendaten")
rain = ut.ts_from_csvs(path_dir)

date_from = dt(2022, 7, 30)
date_to = dt.today() - td(days=1)

rain = ut.ts_filter(rain, date_from=date_from, date_to=None)
rain['kumulativ'] = rain.cumsum()

# Read Niveau Splittfilter

In [ ]:
path_niveau_filter = Path(r"H:\2 Projekte\8000-\8500er\8596\8596.35-Hagnau\05 Berechnungen Grundlagen\Datenauswertung"
    r"\Daten\Daten-SABA_1min_220701-230731.csv")

date_parser = lambda x: pd.to_datetime(x, format=date_format)
niveau_filter = pd.read_csv(path_niveau_filter, sep=';')
niveau_filter.insert(0, 'Datum', 0)
niveau_filter.iloc[:,3] = niveau_filter.iloc[:,3].str.replace(':', '')
niveau_filter['Datum'] = (niveau_filter.iloc[:,1] + niveau_filter.iloc[:,2]
    + niveau_filter.iloc[:,3])
niveau_filter.drop(columns=niveau_filter.columns[1:4], inplace=True)
niveau_filter.set_index(niveau_filter.columns[0], inplace=True)
date_format = '%d.%m.%Y%H:%M:%S%z'
niveau_filter.index = pd.to_datetime(niveau_filter.index,
    format=date_format)
niveau_filter = niveau_filter * 100 # m -> cm

# Read Data Messdatenbank

In [ ]:
for k, v in mes_dict.items():
    data = uh.mdb_getdata(v['id'])
    data = data.tz_localize(None)
    mes_dict[k].update({'data': data})

# Plots

## Trübung

In [ ]:
# Trübung Zulauf / Ablauf
save_path = save_dir / 'Trübung_Zulauf.html'

data = mes_dict['Trübung Zulauf']['data']
date_end = data.index[-1]
hp.ply_2y(
    dfs1=data,
    dfs2=ut.ts_filter(rain, date_to=date_end),
    names=['Trübung Zulauf', 'Regen Therwil'],
    xlabel='Zeit',
    ylabels=['Trübung [NTU]', 'Regenintensität [mm/10min]'],
    title='Trübung Zulauf',
    save_path=save_path
)

In [ ]:
# Trübung Ablauf
save_path = save_dir / 'Trübung_Ablauf.html'

data = mes_dict['Trübung Ablauf']['data']
date_end = data.index[-1]
hp.ply_2y(
    dfs1=data,
    dfs2=ut.ts_filter(rain, date_to=date_end),
    names=['Trübung Ablauf', 'Regen Therwil'],
    xlabel='Zeit',
    ylabels=['Trübung [NTU]', 'Regenintensität [mm/10min]'],
    title='Trübung Ablauf',
    save_path=save_path
)

In [ ]:
# Trübung Zulauf / Ablauf
save_path = save_dir / 'Trübung_Zulauf-Ablauf.html'

data1 = mes_dict['Trübung Zulauf']['data']
data2 = mes_dict['Trübung Ablauf']['data']
hp.ply_2y(
    dfs1=[data1, data2],
    dfs2=rain,
    names=['Trübung Zulauf', 'Trübung Ablauf', 'Regen Therwil'],
    xlabel='Zeit',
    ylabels=['Trübung [NTU]', 'Regenintensität [mm/10min]'],
    title='Trübung Zulauf / Ablauf',
    save_path=save_path
)

## Zulauf

In [ ]:
id = 1078
q_zu = uh.mdb_getdata(id)

In [ ]:
# Durchfluss Zulauf
save_path = save_dir / 'Durchfluss_Zulauf.html'

hp.ply_2y(
    q_zu,
    rain,
    names=['Durchfluss Zulauf', 'Regen Therwil'],
    xlabel='Zeit',
    ylabels=['Durchfluss [l/s]', 'Regenintensität [mm/10min]'],
    title='Durchfluss Zulauf',
    save_path=save_path
)

## Messausfälle

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=q_zu.index,
    y=q_zu.iloc[:,0],
    mode='lines+markers',
    line=dict(color='black'),
    marker=dict(symbol='circle', color='cyan', size=2)
))

fig.update_xaxes(title='Zeit');
fig.update_yaxes(title='Durchfluss [l/s]');
fig.update_layout(title='Übersicht Messausfälle');

save_path = save_dir / 'Messausfälle.html'
fig.write_html(save_path)

## Niveau RSB

In [ ]:
# Niveau Retentionssickerbecken
data = mes_dict['Niveau Retentionssickerbecken']['data']
date_end = data.index[-1]
fig = hp.ply_2y(
    dfs1=data,
    dfs2=ut.ts_filter(rain, date_to=date_end),
    names=['Niveau RSB', 'Regen Therwil'],
    xlabel='Zeit',
    ylabels=['Niveau [cm]', 'Regenintensität [mm/10min]'],
    title='Niveau Retentionssickerbecken',
)


save_path = save_dir / 'Niveau_RSB.html'
fig.write_html(save_path)